In [3]:
from LLmsfJiT import read_trec_qrels
import pandas as pd
from sklearn.metrics import confusion_matrix, cohen_kappa_score
from itertools import repeat

## Read Files

In [42]:
human_qrels = read_trec_qrels("../rel_files/rel_table_qrels_sample_100.txt")
df = pd.DataFrame(human_qrels, columns=["Topic", "It", "Doc", "Rel_Human"])

gpt_split = read_trec_qrels("../gpt_judgements/ex_qrels.txt") # split
gpt_split_bin = read_trec_qrels("../gpt_judgements/ex_qrels_bin_sys.txt") # split binary
gpt_split_bin_topic = read_trec_qrels("../gpt_judgements/ex_qrels_bin_topic_in_sys.txt") # split binary topic
gpt_split_topic = read_trec_qrels("../gpt_judgements/ex_qrels_topic_in_sys.txt") # split topic
gpt_bin = read_trec_qrels("../gpt_judgements/ex_qrels_bin_no_sys.txt") # binary 
gpt_bin_topic = read_trec_qrels("../gpt_judgements/ex_qrels_bin_topic.txt") # binary topic 
gpt_topic = read_trec_qrels("../gpt_judgements/ex_qrels_topic.txt") # topic 
gpt = read_trec_qrels("../gpt_judgements/ex_qrels_A.txt") # 

gpt_split_df = pd.DataFrame(gpt_split, columns=["Topic", "It", "Doc", "Rel_GPT_split"])
gpt_split_bin_df = pd.DataFrame(gpt_split_bin, columns=["Topic", "It", "Doc", "Rel_GPT_split_bin"])
gpt_split_bin_topic_df = pd.DataFrame(gpt_split_bin_topic, columns=["Topic", "It", "Doc", "Rel_GPT_split_bin_topic"])
gpt_split_topic_df = pd.DataFrame(gpt_split_topic, columns=["Topic", "It", "Doc", "Rel_GPT_split_topic"])
gpt_bin_df = pd.DataFrame(gpt_bin, columns=["Topic", "It", "Doc", "Rel_GPT_bin"])
gpt_bin_topic_df = pd.DataFrame(gpt_bin_topic, columns=["Topic", "It", "Doc", "Rel_GPT_bin_topic"])
gpt_topic_df = pd.DataFrame(gpt_topic, columns=["Topic", "It", "Doc", "Rel_GPT_topic"])
gpt_df = pd.DataFrame(gpt, columns=["Topic", "It", "Doc", "Rel_GPT"])

df["Rel_Human"] = df["Rel_Human"].astype(int)

df["Rel_GPT_split"] = gpt_split_df["Rel_GPT_split"].astype(int)
df["Rel_GPT_split_bin"] = gpt_split_bin_df["Rel_GPT_split_bin"].astype(int)
df["Rel_GPT_split_bin_topic"] = gpt_split_bin_topic_df["Rel_GPT_split_bin_topic"].astype(int)
df["Rel_GPT_split_topic"] = gpt_split_topic_df["Rel_GPT_split_topic"].astype(int)
df["Rel_GPT_bin"] = gpt_bin_df["Rel_GPT_bin"].astype(int)
df["Rel_GPT_bin_topic"] = gpt_bin_topic_df["Rel_GPT_bin_topic"].astype(int)
df["Rel_GPT_topic"] = gpt_topic_df["Rel_GPT_topic"].astype(int)
df["Rel_GPT"] = gpt_df["Rel_GPT"].astype(int)

df.loc[df["Rel_Human"] == 0, "Rel_Human_bin"] = 0
df.loc[df["Rel_Human"] > 0, "Rel_Human_bin"] = 1
df["Rel_Human_bin"] = df["Rel_Human_bin"].astype(int)

In [43]:
df

,Topic,It,Doc,Rel_Human,Rel_GPT_split,Rel_GPT_split_bin,Rel_GPT_split_bin_topic,Rel_GPT_split_topic,Rel_GPT_bin,Rel_GPT_bin_topic,Rel_GPT_topic,Rel_GPT,Rel_Human_bin
0,55,0,5/1438042988718.8_20150728002308-00148-ip-10-2...,1,1,1,1,1,1,1,1,1,1
1,25,0,11/1438042981460.12_20150728002301-00113-ip-10...,0,0,0,0,0,0,0,0,1,0
2,24,0,13/1438042989891.18_20150728002309-00180-ip-10...,2,1,1,1,1,1,1,1,1,1
3,55,0,39/1438043062723.96_20150728002422-00055-ip-10...,0,1,0,0,0,0,0,0,1,0
4,32,0,48/1438042987775.70_20150728002307-00036-ip-10...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,24,0,32/1438042990603.54_20150728002310-00051-ip-10...,0,1,1,1,1,1,1,1,1,0
96,14,0,5/1438042988718.8_20150728002308-00150-ip-10-2...,0,2,1,1,2,1,1,2,2,0
97,49,0,46/1438042988922.24_20150728002308-00185-ip-10...,0,0,0,0,0,0,0,0,1,0
98,57,0,31/1438042989018.48_20150728002309-00112-ip-10...,0,0,0,0,0,0,0,0,1,0


## $A$ (split)

In [60]:
cm = confusion_matrix(y_true=df["Rel_Human"], y_pred=df["Rel_GPT_split"])
cohens_k = round(cohen_kappa_score(df["Rel_Human"], df["Rel_GPT_split"]), 2)

idx_rows = pd.MultiIndex.from_tuples(list(zip(repeat("WTR Assesors"), sorted(df["Rel_Human"].unique()))))
idx_cols = pd.MultiIndex.from_tuples(list(zip(repeat("GPT-3.5"), sorted(df["Rel_GPT_split"].unique()))))

a = pd.DataFrame(
    data=cm,
    index = idx_rows,
    columns=idx_cols
).T

display(a.T.style.highlight_max(axis=1, props='font-weight:bold'))
print("Cohen's K:", cohens_k)
        # .to_latex(
        #     hrules=True,
        #     multirow_align="t",
        #     caption="Rater agreement between GPT-3.5 and WTR Assesors",
        # )


Cohen's K: 0.2


## $B$ (split | binary)

In [59]:
idx_cols = pd.MultiIndex.from_tuples(list(zip(repeat("WTR Assesors"), sorted(df["Rel_Human"].unique()))))
idx_rows = pd.MultiIndex.from_tuples(list(zip(repeat("GPT-3.5"), sorted(df["Rel_GPT_split_bin"].unique()))))


b = pd.DataFrame(
    confusion_matrix(y_true=df["Rel_Human"], y_pred=df["Rel_GPT_split_bin"]).T[:2, :],   
    index=idx_rows,
    columns=idx_cols
)
display(b.style.highlight_max(axis=1, props='font-weight:bold'))
print("Cohen's K", round(cohen_kappa_score(df["Rel_GPT_split_bin"], df["Rel_Human_bin"]), 2))



Cohen's K 0.32


## $C$ (split | topic)

In [58]:
idx_rows = pd.MultiIndex.from_tuples(list(zip(repeat("WTR Assesors"), sorted(df["Rel_Human"].unique()))))
idx_cols = pd.MultiIndex.from_tuples(list(zip(repeat("GPT-3.5"), sorted(df["Rel_GPT_split_topic"].unique()))))

c = pd.DataFrame(
    confusion_matrix(y_true=df["Rel_Human"], y_pred=df["Rel_GPT_split_topic"]).T,
    index=idx_cols,
    columns=idx_rows
)

display(c.style.highlight_max(axis=1, props='font-weight:bold'))

print("Cohen's K", round(cohen_kappa_score(df["Rel_GPT_split_topic"], df["Rel_Human"]), 2))



Cohen's K 0.25


## $D$ (split | binary | topic)

In [56]:
idx_rows = pd.MultiIndex.from_tuples(list(zip(repeat("WTR Assesors"), sorted(df["Rel_Human"].unique()))))
idx_cols = pd.MultiIndex.from_tuples(list(zip(repeat("GPT-3.5"), sorted(df["Rel_GPT_split_bin_topic"].unique()))))

d = pd.DataFrame(
    confusion_matrix(y_true=df["Rel_Human"], y_pred=df["Rel_GPT_split_bin_topic"]).T[:2, :],
    index=idx_cols,
    columns=idx_rows
)
display(d.style.highlight_max(axis=1, props='font-weight:bold'))
print("Cohen's K", round(cohen_kappa_score(df["Rel_GPT_split_bin_topic"], df["Rel_Human_bin"]), 2))



Cohen's K 0.44


## $A'$ (graded)

In [62]:
idx_rows = pd.MultiIndex.from_tuples(list(zip(repeat("WTR Assesors"), sorted(df["Rel_Human"].unique()))))
idx_cols = pd.MultiIndex.from_tuples(list(zip(repeat("GPT-3.5"), sorted(df["Rel_GPT"].unique()))))

a_ = pd.DataFrame(
    confusion_matrix(y_true=df["Rel_Human"], y_pred=df["Rel_GPT"]).T,
    index=idx_cols,
    columns=idx_rows
)
display(a_.style.highlight_max(axis=1, props='font-weight:bold'))

print("Cohen's K", round(cohen_kappa_score(df["Rel_GPT"], df["Rel_Human"]), 2))

Cohen's K 0.11


## $B'$ (binary)

In [63]:
idx_rows = pd.MultiIndex.from_tuples(list(zip(repeat("WTR Assesors"), sorted(df["Rel_Human"].unique()))))
idx_cols = pd.MultiIndex.from_tuples(list(zip(repeat("GPT-3.5"), sorted(df["Rel_GPT_bin"].unique()))))

b_ = pd.DataFrame(
    confusion_matrix(y_true=df["Rel_Human"], y_pred=df["Rel_GPT_bin"]).T[:2, :],
    index=idx_cols,
    columns=idx_rows
)
display(b_.style.highlight_max(axis=1, props='font-weight:bold'))

print("Cohen's K", round(cohen_kappa_score(df["Rel_GPT_bin"], df["Rel_Human_bin"]), 2))

Cohen's K 0.33


## $C'$ (graded | topic)

In [54]:
idx_rows = pd.MultiIndex.from_tuples(list(zip(repeat("WTR Assesors"), sorted(df["Rel_Human"].unique()))))
idx_cols = pd.MultiIndex.from_tuples(list(zip(repeat("GPT-3.5"), sorted(df["Rel_GPT_topic"].unique()))))

c_ = pd.DataFrame(
    confusion_matrix(y_true=df["Rel_Human"], y_pred=df["Rel_GPT_topic"]).T,
    index=idx_cols,
    columns=idx_rows
)
display(c_.style.highlight_max(axis=1, props='font-weight:bold'))

print("Cohen's K", round(cohen_kappa_score(df["Rel_GPT_topic"], df["Rel_Human"]), 2))

Cohen's K 0.35


## $D'$ (binary | topic)

In [55]:
idx_rows = pd.MultiIndex.from_tuples(list(zip(repeat("WTR Assesors"), sorted(df["Rel_Human"].unique()))))
idx_cols = pd.MultiIndex.from_tuples(list(zip(repeat("GPT-3.5"), sorted(df["Rel_GPT_bin_topic"].unique()))))

d_ = pd.DataFrame(
    confusion_matrix(y_true=df["Rel_Human"], y_pred=df["Rel_GPT_bin_topic"]).T[:2, :],
    index=idx_cols,
    columns=idx_rows
)
display(d_.style.highlight_max(axis=1, props='font-weight:bold'))

print("Cohen's K", round(cohen_kappa_score(df["Rel_GPT_bin_topic"], df["Rel_Human_bin"]), 2))

Cohen's K 0.3


## Baseline Rating(irr, rel, high_rel) mapped to bin

In [10]:
df.loc[df["Rel_GPT"] == 0, "Rel_GPT_bin_mapped"] = 0
df.loc[df["Rel_GPT"] > 0, "Rel_GPT_bin_mapped"] = 1
df["Rel_GPT_bin_mapped"] = df["Rel_GPT_bin_mapped"].astype(int)

idx_rows = pd.MultiIndex.from_tuples(list(zip(repeat("WTR Assesors"), sorted(df["Rel_Human_bin"].unique()))))
idx_cols = pd.MultiIndex.from_tuples(list(zip(repeat("GPT-3.5"), sorted(df["Rel_GPT_bin_mapped"].unique()))))

cm_2 = pd.DataFrame(
    confusion_matrix(y_true=df["Rel_Human_bin"], y_pred=df["Rel_GPT_bin_mapped"]),
    index=idx_rows,
    columns=idx_cols
).T 
display(cm_2.style.highlight_max(axis=1, props='font-weight:bold'))
print("Cohen's K", cohen_kappa_score(df["Rel_GPT_bin_mapped"], df["Rel_Human_bin"]))



Cohen's K 0.2660550458715596


## Baseline Rating no system instructions in binary

In [12]:
idx_rows = pd.MultiIndex.from_tuples(list(zip(repeat("WTR Assesors"), sorted(df["Rel_Human_bin"].unique()))))
idx_cols = pd.MultiIndex.from_tuples(list(zip(repeat("GPT-3.5"), sorted(df["Rel_GPT_bin_no_sys"].unique()))))

display(pd.DataFrame(
    confusion_matrix(y_true=df["Rel_Human_bin"], y_pred=df["Rel_GPT_bin_no_sys"]),
    index=idx_rows,
    columns=idx_cols
).T.style.highlight_max(axis=1, props='font-weight:bold'))

print("Cohen's K", cohen_kappa_score(df["Rel_GPT_bin_no_sys"], df["Rel_Human_bin"]))


Cohen's K 0.33194154488517746
